In [195]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', 1000)
# 무시
pd.set_option('mode.chained_assignment',  None)
import numpy as np
import seaborn as sns # 시각화 라이브러리
from matplotlib import rc
rc('font',family='Malgun Gothic')
import math

In [196]:
def data_process(columns, data, start, end):
    categorys = data[columns].value_counts()[start:end]
    
    category_list = []
    for category in categorys.index:
        com = data[data.loc[:,columns] == category]
        category_list.append(com)

    spec_data = category_list.copy()
    spec_num = 0
    for spec in spec_data:
        spec.reset_index(inplace=True)
        spec.drop('index',axis=1,inplace=True)
        spec_num += len(spec)
    print('총 스펙개수 : {}'.format(spec_num))

    new_data = pd.concat(spec_data)
    new_data.reset_index(inplace=True)
    new_data.drop('index',axis=1,inplace=True)
    return new_data

In [197]:
# data = pd.read_csv("데이터 전처리 통합.csv", encoding="EUC-KR", index_col=0)
# data = pd.read_csv("데이터 고졸 제거.csv", encoding="EUC-KR", index_col=0)
data = pd.read_csv("데이터 분기별 제거.csv", encoding="EUC-KR", index_col=0)
data_learn = data.drop(['분기','부서','학점','스펙지수'],axis=1)
data_learn['부서'] = data['부서']

In [198]:
# data_learn = data_process('기업정보',data_learn, 0, 1)
# data_learn = data_process('합격회사',data_learn, 2, 3)
data_learn = data_process('부서',data_learn, 0, 5)

총 스펙개수 : 14032


In [199]:
# X = data.iloc[:,[2,5,6,7,8,16]].copy()
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
X = data_learn.iloc[:,2:13].copy()
X = scaler.fit_transform(X)
y = data_learn.iloc[:,14].copy()

In [200]:
#라벨 인코딩
y_label = y.copy()
indexdata = range(0,len(y_label.unique()))
imsiSeries = pd.Series(data=indexdata, index=y_label.unique())
y = y_label.map(imsiSeries)

In [201]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True,random_state=34)

In [202]:
data_learn

,합격회사,기업정보,토익,토익스피킹,OPIC,외국어(기타),자격증,해외경험,인턴,수상내역,교내/사회/봉사,최종학력,전공,학점등급,부서
0,(주)신원,섬유·의류·패션,0,5,0,0,3,1,0,0,0,2,4,0,직무무관
1,(주)신원,섬유·의류·패션,890,0,0,0,0,1,0,2,0,3,4,2,직무무관
2,광동제약(주),제약·보건·바이오,890,0,0,0,3,1,0,0,0,4,4,2,직무무관
3,(주)LG화학,화학·에너지·환경,975,8,0,0,0,0,0,2,0,3,4,0,직무무관
4,(주)LG화학,화학·에너지·환경,870,6,0,0,1,0,0,0,4,3,4,2,직무무관
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14027,시원스쿨,학원·어학원·교육원,0,0,0,0,0,0,0,0,0,4,6,0,마케팅
14028,시원스쿨,학원·어학원·교육원,700,0,0,0,4,0,0,0,0,2,10,2,마케팅
14029,포탈하이웨이,백화점·유통·도소매,0,0,0,0,4,0,0,0,3,2,3,3,마케팅
14030,초록우산어린이재단,공기업·공공기관,960,0,8,0,0,0,0,0,0,3,10,0,마케팅


In [203]:
len(y.unique())

5

In [204]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score , plot_roc_curve, accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import xgboost as xgb

rf = RandomForestClassifier()
ad = AdaBoostClassifier(base_estimator =rf)
dt = DecisionTreeClassifier()
kn = KNeighborsClassifier()
lr = LogisticRegression()
rbf = RBF()
gp = GaussianProcessClassifier(1.0 * RBF(1.0))
mlp = MLPClassifier(alpha=1, max_iter=1000)
gnb = GaussianNB()
svc = SVC(random_state = 43, C = 10, gamma = 0.1, kernel ='rbf')

models = [rf, lr, dt, kn, svc, mlp, gnb]

In [205]:
for model in models:
    model.fit(X_test, y_test)
#     pre = model.predict(X_test)
    scores = cross_val_score(model, X_test, y_test, cv=5).mean().round(3)
#     f1score = metrics.f1_score(y_test, y_pred).round(3)
    print(model, '\n', 'Accuracy:', scores, '\n')

RandomForestClassifier()

RandomForestClassifier() 
 Accuracy: 0.685 



LogisticRegression()

C:\Users\SM2143\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression() 
 Accuracy: 0.558 



...

SVC(C=10, gamma=0.1, random_state=43) 
 Accuracy: 0.677 



MLPClassifier(alpha=1, max_iter=1000)

MLPClassifier(alpha=1, max_iter=1000) 
 Accuracy: 0.67 



GaussianNB()

GaussianNB() 
 Accuracy: 0.53 



In [197]:
model = mlp
model.fit(X_test, y_test)
scores = cross_val_score(model, X_test, y_test, cv=5).mean().round(3)
print(model, '\n', 'Accuracy:', scores, '\n')

C:\Users\SM2143\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(alpha=1, max_iter=1000)

C:\Users\SM2143\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


MLPClassifier(alpha=1, max_iter=1000) 
 Accuracy: 0.482 



C:\Users\SM2143\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(
